# About some q-stuff$\def\qbinom#1#2{\left[\begin{array}{c}#1\\#2\end{array}\right]_{q}}$

In [1]:
%display latex
R.<q> = QQ[]

In [2]:
def qpochammer(a, n : int, q=q):
    '''Method for obtaining (a;q)_n'''
    return prod(1-a*q^k for k in range(n))

def qfactorial(n:int, q=q):
    '''Method for obtaining [n]_q! = (q;q)_n/(1-q)^n'''
    return qpochammer(q,n, q=q)/(1-q)^n

def qbinomial(n:int, m:int, q=q):
    '''Method for obtaining [n,m]_q = [n]_q!/([m]_q![n-m]_q!)'''
    if m < 0 or m > n:
        return 0
    return qfactorial(n,q=q)/(qfactorial(m, q=q)*qfactorial(n-m, q=q))

The sequences that we are interested in work on the polynomials on $q$. This means we have to consider the sequences 
$$\left(\mathbb{K}(q)\right)^{\mathbb{N}}.$$

In this ring of sequences, the basis are sequences themselves $((a_{n,k})_k)_n$:

In [3]:
canonical = lambda n : lambda k : kronecker_delta(n,k)
Matrix([[canonical(n)(k) for k in range(10)] for n in range(5)])

[1 0 0 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0]

Since we are working over an extension of $\mathbb{K}$ all the usual bases still work on this ring:

In [4]:
power = lambda n : lambda k : k^n # this is a factorial basis
Matrix([[power(n)(k) for k in range(10)] for n in range(5)])

[   1    1    1    1    1    1    1    1    1    1]
[   0    1    2    3    4    5    6    7    8    9]
[   0    1    4    9   16   25   36   49   64   81]
[   0    1    8   27   64  125  216  343  512  729]
[   0    1   16   81  256  625 1296 2401 4096 6561]

In [5]:
binomial_bas = lambda n : lambda k : binomial(k,n) # this is a factorial basis
Matrix([[binomial_bas(n)(k) for k in range(10)] for n in range(5)])

[  1   1   1   1   1   1   1   1   1   1]
[  0   1   2   3   4   5   6   7   8   9]
[  0   0   1   3   6  10  15  21  28  36]
[  0   0   0   1   4  10  20  35  56  84]
[  0   0   0   0   1   5  15  35  70 126]

## A $q$-analog
Can we try to have a real $q$-basis? Using the $q$-binomial, we obtain the following:

In [24]:
qbinomial_bas = lambda n : lambda k : qbinomial(k,n) # this is a factorial basis
Matrix([[qbinomial_bas(n)(k) for k in range(6)] for n in range(4)])

[                                        1                                         1                                         1                                         1                                         1                                         1]
[                                        0                                         1                                     q + 1                               q^2 + q + 1                         q^3 + q^2 + q + 1                   q^4 + q^3 + q^2 + q + 1]
[                                        0                                         0                                         1                               q^2 + q + 1                 q^4 + q^3 + 2*q^2 + q + 1 q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1]
[                                        0                                         0                                         0                                         1                         q^3 + q^2 + q + 1 q^6 + q^5 + 2*q^4 + 2*q^3 + 2*q^2 + q + 1]

We have a basis that looks pretty similar to the binomial basis we were considering before. Is this a polynomial basis? Is this a factorial basis?

If we think in similar terms of the binomial, we need to find a sequence of polynomials $P_n(x) \in \mathbb{K}(q)[x]$ such that 
$$P_n(k) = \qbinom{k}{n}$$

By the structure of the basis, these polynomials have as roots $(0,1,...)$, so that would mean that they would be a factorial basis. That simply can not happen:

In [7]:
QB1 = qbinomial_bas(1) # should be a polynomial of degree 1: a(q) + b(q)x
[QB1(i) for i in range(10)] # the term 0 means a(q) = 0, and the term 1 means b(q) = 1... which can not be.

[0,
 1,
 q + 1,
 q^2 + q + 1,
 q^3 + q^2 + q + 1,
 q^4 + q^3 + q^2 + q + 1,
 q^5 + q^4 + q^3 + q^2 + q + 1,
 q^6 + q^5 + q^4 + q^3 + q^2 + q + 1,
 q^7 + q^6 + q^5 + q^4 + q^3 + q^2 + q + 1,
 q^8 + q^7 + q^6 + q^5 + q^4 + q^3 + q^2 + q + 1]

However, we have a sequence in a ring, we have a basis of the ring. We can still study the compatibility with operators:

##### Compatibility with the shift operator $k \rightarrow k+1$

We can find in the literature that the $q$-binomial satisfies the following recurrence equation:
$$\qbinom{k+1}{n} = q^n\qbinom{k}{n} + \qbinom{k}{n-1}.$$

In [8]:
shifted_basis = lambda n : lambda k : qbinomial_bas(n)(k+1)
Matrix([[shifted_basis(n)(k) for k in range(4)] for n in range(3)])

[                        1                         1                         1                         1]
[                        1                     q + 1               q^2 + q + 1         q^3 + q^2 + q + 1]
[                        0                         1               q^2 + q + 1 q^4 + q^3 + 2*q^2 + q + 1]

In [9]:
f = lambda n : lambda k : shifted_basis(n)(k) - qbinomial_bas(n-1)(k) - (q^n)*qbinomial_bas(n)(k)
Matrix([[f(n)(k) for k in range(10)] for n in range(10)])

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]

In the terms of compatibility, we have then that the $q$-binomial basis is compatible with $S: k \mapsto k+1$ with the compatiblity equation:
$$S \cdot P_n = q^n P_n + P_{n-1}$$

##### Compatibility with the multiplication by $k$

In [10]:
kmult_basis = lambda n : lambda k : k*qbinomial_bas(n)(k)
Matrix([[kmult_basis(n)(k) for k in range(4)] for n in range(3)])

[              0               1               2               3]
[              0               1         2*q + 2 3*q^2 + 3*q + 3]
[              0               0               2 3*q^2 + 3*q + 3]

This operation is not compatible with the $q$-binomial basis since the sequence $(k)_k$ will appear as the first element and it can not be finitely represented in terms of $q$-binomials with a fixed down coefficient.

##### Compatibility with the multiplication by $q$

This is trivial since $q$ is part of the field we are taking the sequences from. Hence the compatibility is trivial:
$$q P_k$$

##### Compatibility with the multiplication by $q^k$

This is the key operator to consider the equalities from Ali. What would be the compatibility rule (if it exists) for the multiplication by this number. The main problem here is to properly define the operator, since $k$ is a coefficient from the basis we are taking.

In [11]:
qmult_basis = lambda n : lambda k : (q^k)*qbinomial_bas(n)(k)
Matrix([[qmult_basis(n)(k) for k in range(4)] for n in range(3)])

[              1               q             q^2             q^3]
[              0               q       q^3 + q^2 q^5 + q^4 + q^3]
[              0               0             q^2 q^5 + q^4 + q^3]

In [12]:
f = lambda n : lambda k : qmult_basis(n)(k) - q^n*qbinomial_bas(n)(k) - (q^(2*n+1) - q^n)*qbinomial_bas(n+1)(k)
Matrix([[f(n)(k) for k in range(10)] for n in range(10)])

[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0]

By playing with the formulas, we have detected the following compatibility formula:

$$ (q^k) \cdot P_n = q^n P_n + (q^{2n+1}-q^n)P_{n+1}$$

##### Compatibility with the inverse shift by $S^{-1}$

Can we get a formula for this?

In [13]:
ishifted_basis = lambda n : lambda k : qbinomial_bas(n)(k-1)
Matrix([[ishifted_basis(n)(k) for k in range(5)] for n in range(4)])

[          0           1           1           1           1]
[          0           0           1       q + 1 q^2 + q + 1]
[          0           0           0           1 q^2 + q + 1]
[          0           0           0           0           1]

In [14]:
f = lambda k : ishifted_basis(0)(k) - shifted_basis(0)(k)
[f(k) for k in range(10)]

[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Adapting this context to the ``ore_algebra`` package 

We have so far compatibility with 3 types of operators:
* Multiplication by the constant sequence $(q)_k$: $q: (a_k)_k \mapsto (qa_k)_k$.
* Multiplication by the power sequence $(q^k)_k$: $Q: (a_k)_k \mapsto (q^ka_k)_k$.
* Shift operator w.r.t. the $k$: $S: (a_k)_k \mapsto (a_{k+1})_k$.

We can see from here that $q$ and $Q$ commute, as much as $q$ and $S$. Moreover, since the multiplæication by $(k)_k$ is **not** compatible with the $q$-binomial basis, then we do not need to consider its operator. Hence we can consider the following ore algebra:

In [15]:
from ore_algebra import *
B.<q,Q> = QQ['q', 'Q']
F = FractionField(B)
OA = OreAlgebra(F, ('S', {Q:q*Q},{}), ('Si', {Q:1/q*Q},{})); S,Si = OA.gens()

In [16]:
[q*S, S*q, q*Q, Q*q, Q*S, S*Q, Si*q, q*Si, Si*Q, Q*Si]

[q*S, q*S, q*Q, q*Q, Q*S, (q*Q)*S, q*Si, q*Si, Q/q*Si, Q*Si]

`Q` is now the multiplication by $q^k$  and `S` is the shift operator. Both were compatible, so we can create the correspondance for the compatibility. It is important to remark that the type of operators that we get are very similar: a combination of shifts w.r.t. $n$ and the multiplication by $q^n$. These new operators can be then put into the same ring of operators. One need to keep track on how many changes are done to keep track of the variables and summations.

In [17]:
ruleS = S + Q
ruleq = q
ruleQ = Q + (Q^2-Q)/q*Si
def compatibility(operator):
    if not operator in OA or operator.degree(Si) > 0:
        raise TypeError
        
    operator = OA(operator)
    coefficients = operator.coefficients()
    ## we evaluate each of the coefficients
    coefficients = [coeff(q=ruleq, Q=ruleQ) for coeff in coefficients]
    ## now we compute the powers of the shift
    monomials = [ruleS**m.degree(S) for m in operator.polynomial().monomials()]
    
    ## we multiply everything together
    result_wo_simpl = sum(coefficients[i]*monomials[i] for i in range(len(coefficients)))
    ## we remove the ivnerse shift
    result = result_wo_simpl.degree(Si) * result_wo_sipl
    
    ## we simplify the products of S and Si
    coefficients = result.coefficients()
    monomials = result.polynomial().monomials()
    monomials = [S^(m.degree(S)-m.degree(Si)) for m in monomials]
    return sum(coefficients[i]*monomials[i] for i in range(len(coefficients)))

In [18]:
L = Q*S^2 - Q/q*S - 1

In [19]:
compatibility(L)

TypeError: unsupported operand parent(s) for /: 'Multivariate Ore algebra in S, Si over Fraction Field of Multivariate Polynomial Ring in q, Q over Rational Field' and 'Multivariate Ore algebra in S, Si over Fraction Field of Multivariate Polynomial Ring in q, Q over Rational Field'

We may need to add also $Q^{-1}$, or restric the type of operators in the input. We also need to see what will happen with the $(q;q)_n$ factor that appeared in the formulas.

Otherwise, the compatibility with the $q$-binomial basis seems to be worked out.

## Using FreeAlgebras?

We can use the structure FreeAlgebra to avoid the problems of programming with Ore Algebras, since they require some extra treatment to work properly due to the non-commutativity of some of the operators and also the appearances of the inverses operators.

What we are working here is with a non-commutative algebra that can be built as a quotient of a free algebra. We can do this an implement the reduction methods following the commutation rules that appear. This will lead to a "cannonical" form of the operator (where the variables appear in a fixed order). The following cell implements precisely the ring of operators that we are interested in, and the method `simplify` computes the canninical form taking into account the simplification of the inverses operators.

In [20]:
FA.<Q, iQ, S, iS> = FreeAlgebra(FractionField(QQ[q]))
q = FA.base()('q')
comms = {iQ*Q: 1, S*iS: 1, S*Q: q*Q*S, S*iQ : (1/q)*iQ*S, iS*Q : (1/q)*Q*iS, iS*iQ : q*iQ*iS}

def is_monomial(element):
    r'''
        Method to check whether an objet is a monomial or not
    '''
    return element.coefficients() == [1]

def degrees(monomial):
    r'''
        Method to compute the degrees of a monomial. If not in cannonical form, the result is not reliable.
    '''
    if not is_monomial(monomial):
        raise TypeError
    
    monoid = monomial.trailing_support() # casting to monoid type
    monoid_list = monoid.to_list()
    gens = monoid.parent().gens()
    return tuple([monoid_list.count(g) for g in gens])

def max_degrees(element):
    r'''
        Method to compute the degrees of a polynoial. It assumes it is in cannonical form
    '''
    all_degs = [degrees(m) for m in element.monomials()]
    all_degs = [[r[i] for r in all_degs] for i in range(len(element.parent().gens()))]
    return tuple([max(r) for r in all_degs])

def monomial_from_degrees(degrees, parent):
    r'''
        Method to compute a monomial from the degrees (in cannonical form)
    '''
    gens = parent.gens()
    if len(gens) != len(degrees):
        raise ValueError("Incorrect size")
        
    return prod(gens[i]**degrees[i] for i in range(len(gens)))

def cannonical(element, commutations):
    r'''
        Method to compute a cannonical form of a element of a free algebra where 
        the commutations rules are povided with a dictionary. If a relation
        does not appear in the dictionary, we assume they commute.
    '''
    if is_monomial(element):
        # We apply the commutation rules
        monoid = element.trailing_support() # casting to monoid type
        monoid_list = monoid.to_list()
        gens = monoid.parent().gens()
        leq = lambda p, q : gens.index(p) <= gens.index(q)
        for i in range(len(monoid_list)-1):
            if not leq(monoid_list[i], monoid_list[i+1]):
                v1, v2 = monoid_list[i], monoid_list[i+1]
                ori = element.parent()(v1*v2); com = element.parent()(v2*v1)
                monoid_list.pop(i)
                monoid_list[i] = commutations.get(ori, com)
                return cannonical(prod([element.parent()(el) for el in monoid_list]), commutations)
        # If we reach this point, the variables in the monomial are sorted, hence the monomial is cannonical
        return element
    # Otherwise, we compute the new element for each term
    coeffs = element.coefficients(False)
    monomials = [cannonical(m, commutations) for m in element.monomials()]
    
    return sum(coeffs[i]*monomials[i] for i in range(len(monomials)))

def simplify(element, commutations, inv = None):
    r'''
        Method to simplify a expression following some commutations and inverses.
    '''
    # First, we compute the cannonical form
    element = cannonical(element, commutations)
    
    # Now we compute the inverses if they are not provided
    inverses = [] if inv is None else inv.copy()
    for rule in commutations:
        if commutations[rule] == 1: # this is an inverse
            inverses.append(rule.variables())
    
    return _simplify(element, inverses)

def _simplify(element, inverses):
    r'''
        Method to simplify a expression following some commutations and inverses.
    
        This method assumes that element is in cannonical form.
    '''
    coefficients = element.coefficients(False)
    monomials = element.monomials()
    red_monomials = []
    for m in monomials:
        deg = list(degrees(m))
        gens = m.parent().gens()
        for (v1,v2) in inverses:
            i1, i2 = gens.index(v1), gens.index(v2)
            m = min(deg[i1],deg[i2])
            deg[i1] -= m; deg[i2] -= m
        red_monomials.append(monomial_from_degrees(deg, element.parent()))
    return sum(coefficients[i]*red_monomials[i] for i in range(len(coefficients)))

Now, what remains is to implement the substitution (or compatibility) rules for all the generators that appear in the Free Algebra. Once we have this, we can always substitue purely in the operator and apply the method `simplify`to obtain the new recurrence equation:

In [21]:
ruleS = S + Q
ruleQ = Q + (Q^2-Q)/q*iS
rules = {'S': ruleS, 'Q': ruleQ}

def evaluate(element, rules):
    r'''
        Method that evaluate a cannonical element with some rules.
        
        This assumes that:
        * ``rules`` cover all the variables appearing in ``element``
        * ``element`` is in cannonical form.
        * The output is **not** simplified.
    '''
    coefficients = element.coefficients(False)
    monomials = [evaluate_monomial(m, rules) for m in element.monomials()]
    
    return sum(coefficients[i]*monomials[i] for i in range(len(coefficients)))

def evaluate_monomial(monomial, rules):
    r'''
        Method that evaluate a cannonical monomial with some rules.
        
        This assumes that:
        * ``rules`` cover all the variables appearing in ``monomial``
        * ``monomial`` is in cannonical form.
    '''
    gens = [str(el) for el in monomial.parent().gens()]; n = len(gens)
    degs = degrees(monomial)
    if any([(not gens[i] in rules) for i in range(n) if degs[i] > 0]):
        raise ValueError("An element is not given by the rules: impossible to substitute")
    
    return prod(rules[gens[i]]**degs[i] for i in range(n) if degs[i] > 0)    

def compatibility(element, rules, commutations, inverses= None):
    return simplify(evaluate(element, rules), commutations, inverses)

def remove_iS(element, commutations, inverses = None):
    i = list(element.parent().gens()).index(iS)
    m = max(degrees(mon)[i] for mon in element.monomials())
    return simplify(S**m * element, commutations, inverses)

In [22]:
L = Q*S^2 - Q/q*S - 1; L

-1 - 1/q*Q*S + Q*S^2

In [23]:
L1 = compatibility(L, rules, comms); L1

-1 + 1/q^2*Q + ((-2*q-2)/q^2)*Q^2 - 2/q*Q*S + ((q^2+q+1)/q^2)*Q^3 + ((q^2+q+1)/q)*Q^2*S + 1/q^3*Q^2*iS + Q*S^2 - 2/q^3*Q^3*iS + 1/q^3*Q^4*iS

### The example by Ali

The example by Ali is a recurrence with plenty of components:

$$\begin{array}{rll}
q^{19 + 6n}\left({\left(q^{24} + q^{23} + q^{22}\right)} q^{6 \, n} - {\left(q^{20} + q^{19} + q^{18}\right)} q^{5 \, n} + {\left(q^{17} + q^{16} + q^{15}\right)} q^{4 \, n} + {\left(q^{12} + q^{11} + q^{10}\right)} q^{3 \, n} - {\left(q^{9} + q^{8} + q^{7}\right)} q^{2 \, n} + {\left(q^{5} + 2 \, q^{4} + q^{3}\right)} q^{n} - 1\right) & f(n) + &  \\
q^{5 \, n + 19}\left({\left(q^{28} + q^{27} + q^{26} + q^{25} + q^{24} + q^{23}\right)} q^{8 \, n} - {\left(q^{21} + q^{20} + q^{19}\right)} q^{7 \, n} + {\left(q^{21} + q^{20} + q^{19} + q^{18} + q^{17} + q^{16}\right)} q^{6 \, n} + {\left(q^{16} + q^{15} + q^{14} + q^{13} + q^{12} + q^{11}\right)} q^{5 \, n} + {\left(q^{15} + q^{14} + q^{13} - q^{10} - q^{9} - q^{8}\right)} q^{4 \, n} - {\left(q^{12} + q^{11} - q^{9} - 2 \, q^{8} - 2 \, q^{7} - 2 \, q^{6} - 2 \, q^{5} - q^{4}\right)} q^{3 \, n} + {\left(q^{8} - q^{5} - q\right)} q^{2 \, n} + {\left(q^{3} + q^{2} + q\right)} q^{n} - 1\right) & f(1 + n) + & \\
q^{3 \, n + 12} \left( {\left(q^{37} + q^{36} + q^{35}\right)} q^{11 \, n} - {\left(q^{33} + q^{32} + q^{31}\right)} q^{10 \, n} + {\left(q^{32} + 2 \, q^{31} + 3 \, q^{30} + 3 \, q^{29} + 2 \, q^{28} + q^{27}\right)} q^{9 \, n} - {\left(q^{30} + 2 \, q^{29} + 3 \, q^{28} + 2 \, q^{27} + 2 \, q^{26} - q^{23}\right)} q^{8 \, n} + {\left(q^{27} + 2 \, q^{26} + 3 \, q^{25} + 3 \, q^{24} + 2 \, q^{23} + q^{22} - q^{21} - q^{20} - q^{19}\right)} q^{7 \, n} - {\left(q^{25} + 2 \, q^{24} + 4 \, q^{23} + 2 \, q^{22} + 2 \, q^{21} - q^{20} - 2 \, q^{19} - 4 \, q^{18} - 4 \, q^{17} - 3 \, q^{16} - q^{15}\right)} q^{6 \, n} + {\left(q^{21} + 2 \, q^{20} + 2 \, q^{19} - q^{17} - 3 \, q^{16} - 2 \, q^{15} - 3 \, q^{14} - 2 \, q^{13} - q^{12}\right)} q^{5 \, n} - {\left(q^{18} + q^{17} + 2 \, q^{16} - q^{14} - 3 \, q^{13} - 3 \, q^{12} - 3 \, q^{11} - 2 \, q^{10} - q^{9}\right)} q^{4 \, n} - {\left(q^{13} + q^{12} + 3 \, q^{11} + 3 \, q^{10} + 3 \, q^{9} + q^{8} + q^{7}\right)} q^{3 \, n} + {\left(q^{10} + q^{9} + 2 \, q^{8} + 2 \, q^{7} + q^{6}\right)} q^{2 \, n} - {\left(q^{6} + 2 \, q^{5} + 3 \, q^{4} + 2 \, q^{3}\right)} q^{n} + q + 1 \right) & f(2 + n) + & \\
q^{2 \, n + 10} \left( {\left(q^{40} + q^{39} + q^{38}\right)} q^{12 \, n} - {\left(q^{37} + 2 \, q^{36} + 2 \, q^{35} + q^{34}\right)} q^{11 \, n} + {\left(q^{35} + q^{34} + 2 \, q^{33} + q^{32} + q^{31}\right)} q^{10 \, n} + {\left(q^{30} + 2 \, q^{29} + 3 \, q^{28} + 2 \, q^{27} + q^{26}\right)} q^{9 \, n} - {\left(q^{29} + q^{28} + 3 \, q^{27} + 4 \, q^{26} + 6 \, q^{25} + 5 \, q^{24} + 3 \, q^{23} + q^{22}\right)} q^{8 \, n} + {\left(q^{26} + 4 \, q^{25} + 5 \, q^{24} + 7 \, q^{23} + 7 \, q^{22} + 7 \, q^{21} + 5 \, q^{20} + 3 \, q^{19} + q^{18}\right)} q^{7 \, n} - {\left(2 \, q^{22} + 3 \, q^{21} + 3 \, q^{20} + 3 \, q^{19} + 2 \, q^{18} + 2 \, q^{17} + q^{16} + q^{15}\right)} q^{6 \, n} + {\left(q^{18} - q^{15} - 2 \, q^{14} - 2 \, q^{13} - q^{12}\right)} q^{5 \, n} + {\left(2 \, q^{15} + 4 \, q^{14} + 7 \, q^{13} + 7 \, q^{12} + 8 \, q^{11} + 6 \, q^{10} + 4 \, q^{9} + q^{8}\right)} q^{4 \, n} - {\left(q^{12} + 3 \, q^{11} + 5 \, q^{10} + 6 \, q^{9} + 5 \, q^{8} + 4 \, q^{7} + 2 \, q^{6} + q^{5}\right)} q^{3 \, n} + {\left(q^{8} + 2 \, q^{7} + 5 \, q^{6} + 5 \, q^{5} + 4 \, q^{4} + 2 \, q^{3} + q^{2}\right)} q^{2 \, n} - q^{n + 2} - 1 \right) & f(3 + n) + & \\
q^{n + 4} - 1 \left( {\left(q^{18} + q^{17} + q^{16}\right)} q^{6 \, n} - {\left(q^{15} + q^{14} + q^{13}\right)} q^{5 \, n} + {\left(q^{13} + q^{12} + q^{11}\right)} q^{4 \, n} + {\left(q^{9} + q^{8} + q^{7}\right)} q^{3 \, n} - {\left(q^{7} + q^{6} + q^{5}\right)} q^{2 \, n} + {\left(q^{4} + 2 \, q^{3} + q^{2}\right)} q^{n} - 1 \right) & f[4 + n] & = 0
\end{array}$$

In [54]:
q,n = var('q,n')
coeffs = [
    (-q^(19 + 6*n) + q^(22 + 7*n) + 2*q^(23 + 7*n) + q^(24 + 7*n) - q^(26 + 8*n) - q^(27 + 8*n) - q^(28 + 8*n) + q^(29 + 9*n) + # f(n)
     q^(30 + 9*n) + q^(31 + 9*n) + q^(34 + 10*n) + q^(35 + 10*n) + q^(36 + 10*n) - q^(37 + 11*n) - q^(38 + 11*n) - q^(39 + 11*n) + 
     q^(41 + 12*n) + q^(42 + 12*n) + q^(43 + 12*n)), 
   (-q^(19 + 5*n) + q^(20 + 6*n) + q^(21 + 6*n) + q^(22 + 6*n) - q^(20 + 7*n) - q^(24 + 7*n) + q^(27 + 7*n) + q^(23 + 8*n) + # f(n+1)
     2*q^(24 + 8*n) + 2*q^(25 + 8*n) + 2*q^(26 + 8*n) + 2*q^(27 + 8*n) + q^(28 + 8*n) - q^(30 + 8*n) - q^(31 + 8*n) - q^(27 + 9*n) - 
     q^(28 + 9*n) - q^(29 + 9*n) + q^(32 + 9*n) + q^(33 + 9*n) + q^(34 + 9*n) + q^(30 + 10*n) + q^(31 + 10*n) + q^(32 + 10*n) + 
     q^(33 + 10*n) + q^(34 + 10*n) + q^(35 + 10*n) + q^(35 + 11*n) + q^(36 + 11*n) + q^(37 + 11*n) + q^(38 + 11*n) + q^(39 + 11*n) + 
     q^(40 + 11*n) - q^(38 + 12*n) - q^(39 + 12*n) - q^(40 + 12*n) + q^(42 + 13*n) + q^(43 + 13*n) + q^(44 + 13*n) + q^(45 + 13*n) + 
     q^(46 + 13*n) + q^(47 + 13*n)), 
   (q^(12 + 3*n) + q^(13 + 3*n) - 2*q^(15 + 4*n) - 3*q^(16 + 4*n) - 2*q^(17 + 4*n) - q^(18 + 4*n) + q^(18 + 5*n) + 2*q^(19 + 5*n) + # f(n+2)
     2*q^(20 + 5*n) + q^(21 + 5*n) + q^(22 + 5*n) - q^(19 + 6*n) - q^(20 + 6*n) - 3*q^(21 + 6*n) - 3*q^(22 + 6*n) - 3*q^(23 + 6*n) - 
     q^(24 + 6*n) - q^(25 + 6*n) + q^(21 + 7*n) + 2*q^(22 + 7*n) + 3*q^(23 + 7*n) + 3*q^(24 + 7*n) + 3*q^(25 + 7*n) + q^(26 + 7*n) - 
     2*q^(28 + 7*n) - q^(29 + 7*n) - q^(30 + 7*n) - q^(24 + 8*n) - 2*q^(25 + 8*n) - 3*q^(26 + 8*n) - 2*q^(27 + 8*n) - 3*q^(28 + 8*n) - 
     q^(29 + 8*n) + 2*q^(31 + 8*n) + 2*q^(32 + 8*n) + q^(33 + 8*n) + q^(27 + 9*n) + 3*q^(28 + 9*n) + 4*q^(29 + 9*n) + 4*q^(30 + 9*n) + 
     2*q^(31 + 9*n) + q^(32 + 9*n) - 2*q^(33 + 9*n) - 2*q^(34 + 9*n) - 4*q^(35 + 9*n) - 2*q^(36 + 9*n) - q^(37 + 9*n) - q^(31 + 10*n) - 
     q^(32 + 10*n) - q^(33 + 10*n) + q^(34 + 10*n) + 2*q^(35 + 10*n) + 3*q^(36 + 10*n) + 3*q^(37 + 10*n) + 2*q^(38 + 10*n) + q^(39 + 10*n) + 
     q^(35 + 11*n) - 2*q^(38 + 11*n) - 2*q^(39 + 11*n) - 3*q^(40 + 11*n) - 2*q^(41 + 11*n) - q^(42 + 11*n) + q^(39 + 12*n) + 2*q^(40 + 12*n) + 
     3*q^(41 + 12*n) + 3*q^(42 + 12*n) + 2*q^(43 + 12*n) + q^(44 + 12*n) - q^(43 + 13*n) - q^(44 + 13*n) - q^(45 + 13*n) + q^(47 + 14*n) + 
     q^(48 + 14*n) + q^(49 + 14*n)), 
   (q^(10 + 2*n) + q^(12 + 3*n) - q^(12 + 4*n) - 2*q^(13 + 4*n) - 4*q^(14 + 4*n) - 5*q^(15 + 4*n) - 5*q^(16 + 4*n) - 2*q^(17 + 4*n) - # f(n+3)
     q^(18 + 4*n) + q^(15 + 5*n) + 2*q^(16 + 5*n) + 4*q^(17 + 5*n) + 5*q^(18 + 5*n) + 6*q^(19 + 5*n) + 5*q^(20 + 5*n) + 3*q^(21 + 5*n) + 
     q^(22 + 5*n) - q^(18 + 6*n) - 4*q^(19 + 6*n) - 6*q^(20 + 6*n) - 8*q^(21 + 6*n) - 7*q^(22 + 6*n) - 7*q^(23 + 6*n) - 4*q^(24 + 6*n) - 
     2*q^(25 + 6*n) + q^(22 + 7*n) + 2*q^(23 + 7*n) + 2*q^(24 + 7*n) + q^(25 + 7*n) - q^(28 + 7*n) + q^(25 + 8*n) + q^(26 + 8*n) + 
     2*q^(27 + 8*n) + 2*q^(28 + 8*n) + 3*q^(29 + 8*n) + 3*q^(30 + 8*n) + 3*q^(31 + 8*n) + 2*q^(32 + 8*n) - q^(28 + 9*n) - 3*q^(29 + 9*n) - 
     5*q^(30 + 9*n) - 7*q^(31 + 9*n) - 7*q^(32 + 9*n) - 7*q^(33 + 9*n) - 5*q^(34 + 9*n) - 4*q^(35 + 9*n) - q^(36 + 9*n) + q^(32 + 10*n) + 
     3*q^(33 + 10*n) + 5*q^(34 + 10*n) + 6*q^(35 + 10*n) + 4*q^(36 + 10*n) + 3*q^(37 + 10*n) + q^(38 + 10*n) + q^(39 + 10*n) - q^(36 + 11*n) - 
     2*q^(37 + 11*n) - 3*q^(38 + 11*n) - 2*q^(39 + 11*n) - q^(40 + 11*n) - q^(41 + 12*n) - q^(42 + 12*n) - 2*q^(43 + 12*n) - q^(44 + 12*n) - 
     q^(45 + 12*n) + q^(44 + 13*n) + 2*q^(45 + 13*n) + 2*q^(46 + 13*n) + q^(47 + 13*n) - q^(48 + 14*n) - q^(49 + 14*n) - q^(50 + 14*n)),
    (-1 + q^(2 + n) + 2*q^(3 + n) + 2*q^(4 + n) - q^(5 + 2*n) - 2*q^(6 + 2*n) - 3*q^(7 + 2*n) - q^(8 + 2*n) + q^(7 + 3*n) + q^(8 + 3*n) + # f(n+4)
     2*q^(9 + 3*n) + q^(10 + 3*n) + q^(11 + 3*n) - q^(13 + 5*n) - q^(14 + 5*n) - 2*q^(15 + 5*n) - q^(16 + 5*n) - q^(17 + 5*n) + q^(16 + 6*n) + 
     2*q^(17 + 6*n) + 2*q^(18 + 6*n) + q^(19 + 6*n) - q^(20 + 7*n) - q^(21 + 7*n) - q^(22 + 7*n))
]

In [63]:
def coeff_to_operator(coeff, dest_q, dest_Q):
    import re
    opers = coeff.factor().operands()
    if len(opers) == 2:
        big, small = opers
        extra = 1
    else:
        big, small, extra = opers
    string_big = str(big.simplify_full())
    string_small = str(small.simplify_full())
    regs = [(r"q\^\((\d*)\*n\)", r"Q^(\1)"),
            (r"q\^\((\d*)\*n \+ (\d*)\)", r"q^(\2)*Q^(\1)"),
            (r"q\^\(n \+ (\d*)\)", r"q^(\1)*Q^(1)"),
            (r"q\^n", r"Q^(1)"),
            (r"\^", r"**")]
    def apply_regs(regs, string):
        for reg in regs:
            string = re.sub(*reg, string)
        return string
    
    q = dest_q; Q = dest_Q
    return dest_Q.parent()(extra)*eval(apply_regs(regs, string_big)) * eval(apply_regs(regs, string_small))

In [64]:
L = sum([coeff_to_operator(coeffs[i], FA(q), FA(Q))*S^i for i in range(len(coeffs))]);
max_degrees(L)

(14, 0, 4, 0)

In [29]:
L

-S^4 + (q^13+q^12)*Q^3*S^2 + q^10*Q^2*S^3 + (2*q^4+2*q^3+q^2)*Q*S^4 - q^19*Q^6 - q^19*Q^5*S + (-q^18-2*q^17-3*q^16-2*q^15)*Q^4*S^2 + q^12*Q^3*S^3 + (-q^8-3*q^7-2*q^6-q^5)*Q^2*S^4 + (q^24+2*q^23+q^22)*Q^7 + (q^22+q^21+q^20)*Q^6*S + (q^22+q^21+2*q^20+2*q^19+q^18)*Q^5*S^2 + (-q^18-2*q^17-5*q^16-5*q^15-4*q^14-2*q^13-q^12)*Q^4*S^3 + (q^11+q^10+2*q^9+q^8+q^7)*Q^3*S^4 + (-q^28-q^27-q^26)*Q^8 + (q^27-q^24-q^20)*Q^7*S + (-q^25-q^24-3*q^23-3*q^22-3*q^21-q^20-q^19)*Q^6*S^2 + (q^22+3*q^21+5*q^20+6*q^19+5*q^18+4*q^17+2*q^16+q^15)*Q^5*S^3 + (q^31+q^30+q^29)*Q^9 + (-q^31-q^30+q^28+2*q^27+2*q^26+2*q^25+2*q^24+q^23)*Q^8*S + (-q^30-q^29-2*q^28+q^26+3*q^25+3*q^24+3*q^23+2*q^22+q^21)*Q^7*S^2 + (-2*q^25-4*q^24-7*q^23-7*q^22-8*q^21-6*q^20-4*q^19-q^18)*Q^6*S^3 + (-q^17-q^16-2*q^15-q^14-q^13)*Q^5*S^4 + (q^36+q^35+q^34)*Q^10 + (q^34+q^33+q^32-q^29-q^28-q^27)*Q^9*S + (q^33+2*q^32+2*q^31-q^29-3*q^28-2*q^27-3*q^26-2*q^25-q^24)*Q^8*S^2 + (-q^28+q^25+2*q^24+2*q^23+q^22)*Q^7*S^3 + (q^19+2*q^18+2*q^17+q^16)*Q^6*S^4 + (-q^39-q^38-q^37)*Q^11 + (q^35+q^34+q^33+q^32+q^31+q^30)*Q^10*S + (-q^37-2*q^36-4*q^35-2*q^34-2*q^33+q^32+2*q^31+4*q^30+4*q^29+3*q^28+q^27)*Q^9*S^2 + (2*q^32+3*q^31+3*q^30+3*q^29+2*q^28+2*q^27+q^26+q^25)*Q^8*S^3 + (-q^22-q^21-q^20)*Q^7*S^4 + (q^43+q^42+q^41)*Q^12 + (q^40+q^39+q^38+q^37+q^36+q^35)*Q^11*S + (q^39+2*q^38+3*q^37+3*q^36+2*q^35+q^34-q^33-q^32-q^31)*Q^10*S^2 + (-q^36-4*q^35-5*q^34-7*q^33-7*q^32-7*q^31-5*q^30-3*q^29-q^28)*Q^9*S^3 + (-q^40-q^39-q^38)*Q^12*S + (-q^42-2*q^41-3*q^40-2*q^39-2*q^38+q^35)*Q^11*S^2 + (q^39+q^38+3*q^37+4*q^36+6*q^35+5*q^34+3*q^33+q^32)*Q^10*S^3 + (q^47+q^46+q^45+q^44+q^43+q^42)*Q^13*S + (q^44+2*q^43+3*q^42+3*q^41+2*q^40+q^39)*Q^12*S^2 + (-q^40-2*q^39-3*q^38-2*q^37-q^36)*Q^11*S^3 + (-q^45-q^44-q^43)*Q^13*S^2 + (-q^45-q^44-2*q^43-q^42-q^41)*Q^12*S^3 + (q^49+q^48+q^47)*Q^14*S^2 + (q^47+2*q^46+2*q^45+q^44)*Q^13*S^3 + (-q^50-q^49-q^48)*Q^14*S^3

In [354]:
L1 = compatibility(L, rules, comms); L1

KeyboardInterrupt: 

Second (and smaller example):

In [65]:
coeffs2 = [
    q^(3*n + 6)*(1 - q^(1 + n))*(q + q^(3*n + 6) - q^(3 + n)),
    q^(-3 + n)*(-q^7 + q^(8*n + 16) + q^(8 + n) + q^(9 + n) + q^(8 + 2*n) +
        q^(9 + 2*n) - q^(9 + 3*n) - 2*q^(10 + 3*n) - 2*q^(11 + 3*n) - 
        q^(12 + 3*n) + q^(11 + 4*n) + q^(12 + 4*n) + q^(13 + 4*n) + 
        q^(11 + 5*n) + q^(12 + 5*n) + q^(13 + 5*n) + q^(14 + 5*n) - 
        q^(13 + 6*n) - q^(14 + 6*n) - q^(15 + 6*n)),
    -(q^4 + q^(3*(n+2)) - q^(5 + n))    
]

In [67]:
LL = sum([coeff_to_operator(coeffs2[i], FA(q), FA(Q))*S^i for i in range(len(coeffs2))]);
max_degrees(L)

ValueError: too many values to unpack (expected 3)

In [42]:
%debug

> /home/qh82kz/git/pseries_basis/notebooks/sage/structure/coerce.pyx(1248)sage.structure.coerce.CoercionModel.bin_op()

ipdb> up
> /home/qh82kz/git/pseries_basis/notebooks/sage/structure/element.pyx(1516)sage.structure.element.Element.__mul__()

ipdb> 
> <string>(1)<module>()

ipdb> 
> /tmp/ipykernel_1176/2062720291.py(18)<listcomp>()
     15 
     16     q = dest_q; Q = dest_Q
     17 
---> 18     transformed = [eval(apply_regs(regs, part)) for part in strings]
     19     return prod(transformed)

ipdb> print(q)
q
ipdb> type(q)
<class 'sage.symbolic.expression.Expression'>
ipdb> print(dest_q)
*** NameError: name 'dest_q' is not defined
ipdb> up
> /tmp/ipykernel_1176/2062720291.py(18)coeff_to_operator()
     15 
     16     q = dest_q; Q = dest_Q
     17 
---> 18     transformed = [eval(apply_regs(regs, part)) for part in strings]
     19     return prod(transformed)

ipdb> print(q)
q
ipdb> type(q)
<class 'sage.algebras.free_algebra.FreeAlgebra_generic_with_category.element_class'>
ip

In [49]:
eval?